<center>
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates how to copy an AMI from one region to multiple AWS regions using unSkript actions.</b>
</div>

<br>
</center>
<center><h2>Copy AMI To All Given AWS Regions</h2></center>

# Steps Overview
   1)[Get All AWS Regions](#1)</br>
   2)[Get AMI Name](#2)</br>
   3)[Copy AMI To Other AWS Regions](#3)

### <a id="1"> Get All AWS Regions</a>
Here in step-1 we will collect all the regions from the AWS by using a command `aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text`. This action only executes when the user doesn't provide the destination regions.

>Input parameters: `aws_command`

>Output variable: `regions`

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not destination_regions",
    "condition_result": true
    }''')
task.configure(outputName="regions")
task.configure(inputParamsJson='''{
    "aws_command": "\\"aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text\\""
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

### Modify Region Output
In this action, we convert the output from step 1 to a python list, which will be consumed in subsequent cells.

>Output variable: `destination_regions`

In [ ]:
try:
    if regions:
        result_op = list(regions.split("\n"))
        destination_regions = [x for x in result_op if x != '']
except Exception as e:
    pass

### <a id="2"> Get AMI Name</a>
Here in step-2 we will findout the name of AMI using the given AMI Id with `aws ec2 describe-images --region "+ source_region +" --image-ids "+ ami_id +" --query 'Images[*].[Name]' --output text`.

>Input parameters: `aws_command`

>Output variable: `ami_name`

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_command": "\\"aws ec2 describe-images --region \\"+ source_region +\\" --image-ids \\"+ ami_id +\\" --query 'Images[*].[Name]' --output text\\""
    }''')
task.configure(outputName="ami_name")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

### Create Batch Command to Copy AMI
In this action, we create the batch command to copy AMI to the given region(s) and pass the output to step 3 as iterator.

>Output variable: `aws_command`

In [ ]:
aws_command = []
copy_command = "aws ec2 copy-image --source-image-id {AMI_Id} --source-region {SourceRegion} --region {DestinationRegion} --name {Name}"
if destination_regions:
    for region in destination_regions:
        command = copy_command.format(AMI_Id=ami_id, SourceRegion=source_region,
                                     DestinationRegion=region, Name=ami_name.replace('\n',''))
        aws_command.append(command)

### <a id="3"> Copy AMI To Other AWS Regions</a>
In this action, we will provide all the necessary inputs to copy the AMI from one region to another. If the user did not provide the destination region, we use the output from step 1 and copy the AMI to all destination regions by using `aws ec2 copy-image --source-image-id <ami-id> --source-region <source-region> --region <dest-region> --name <ami-name>`.

>Input parameters: `aws_command`

>Output variable: `command_output`

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "aws_command",
    "iter_parameter": "aws_command"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": false,
    "condition_cfg": "abcd",
    "condition_result": true
    }''')
task.configure(outputName="command_output")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

   In this Runbook, we demonstrated the use of unSkript's AWS actions to perform copy AMI from one region to multiple AWS regions.
   To view the full platform capabilities of unSkript please visit https://us.app.unskript.io